In [1]:
import os
%pwd

'f:\\ml projects\\chicken_disease_classification\\notebooks'

In [2]:
os.chdir('../')
%pwd

'f:\\ml projects\\chicken_disease_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [4]:
from chicken_disease_classifier.utils.common import create_directories,read_yaml
from chicken_disease_classifier.constants import *

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks

        create_directories([Path(config.root_dir),Path(config.tensorboard_root_log_dir),Path(os.path.dirname(config.checkpoint_model_filepath))])

        preprare_callback_config = PrepareCallbacksConfig(
            root_dir= config.root_dir,
            tensorboard_root_log_dir= config.tensorboard_root_log_dir,
            checkpoint_model_filepath= config.checkpoint_model_filepath
        )

        return  preprare_callback_config


In [5]:
import os
from urllib import request
from zipfile import ZipFile
import tensorflow as tf
import time

In [6]:
class PrepareCallback:
    def __init__(self,config:PrepareCallbacksConfig) -> None:
        self.config = config

    @property
    def _create_tensorboard(self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,f"tb_logs_at_{timestamp}")

        return tf.keras.callbacks.TensorBoard(log_dir= tb_running_log_dir)
    
    @property
    def _create_checkpoint_model(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.checkpoint_model_filepath,save_best_only=True)
    
    def create_ckpt_callbacks(self):
        return [self._create_tensorboard,self._create_checkpoint_model]

In [7]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callbacks_config()
    prepare_callback = PrepareCallback(prepare_callback_config)
    callbac_list = prepare_callback.create_ckpt_callbacks()
except Exception as e:
    raise e

[ 2023-11-22 18:26:02,625 ] 26 - common - INFO - <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'> Loaded Successfully
[ 2023-11-22 18:26:02,635 ] 26 - common - INFO - <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> Loaded Successfully
[ 2023-11-22 18:26:02,648 ] 48 - common - INFO - Created Directory at : artifacts
[ 2023-11-22 18:26:02,651 ] 48 - common - INFO - Created Directory at : artifacts\prepare_callbacks
[ 2023-11-22 18:26:02,655 ] 48 - common - INFO - Created Directory at : artifacts\prepare_callbacks\tensorboard_log_dir
[ 2023-11-22 18:26:02,658 ] 48 - common - INFO - Created Directory at : artifacts\prepare_callbacks\checkpoint_dir
